In [6]:
import pikepdf
import pdfplumber
import re
import datetime
import os
from utils.post_processing import merge_rename_and_summate
from utils.extraction_handler import extract_text_from_pdf_page, extract_info_from_text
import subprocess


In [8]:
# subprocess.run(["../scripts/delete_pdf_files.sh"], shell=True)
# print(f'===========================================================================================')


file_path = '/Users/ekim/workspace/txb/docs/ccm_full.pdf'
temp_dir = r'/Users/ekim/workspace/txb/mock/K-Drive/DTN Reports/Credit Cards/EXXONMOBIL (10005)/temp'


company_names = ['VALERO', 'CONCORD FIRST DATA RETRIEVAL', 'EXXONMOBIL', 'U.S. OIL COMPANY', 'DK Trading & Supply', 'CVR SUPPLY & TRADING, LLC']

regex_patterns = ['EFT-\d+', 'CMB-\d+', 'CCM-\d+', 'CMB-\d+', 'RTV-\d+', 'CBK-\+']


company_name_to_subdir_full_path_mapping_credit_cards = {

    'VALERO': r'/Users/ekim/workspace/txb/mock/K-Drive/DTN Reports/Credit Cards/Valero (10006)',

    'CONCORD FIRST DATA RETRIEVAL': r'/Users/ekim/workspace/txb/mock/K-Drive/DTN Reports/Credit Cards/First Data',

    'EXXONMOBIL': r'/Users/ekim/workspace/txb/mock/K-Drive/DTN Reports/Credit Cards/EXXONMOBIL (10005)'

}



In [9]:
def create_and_save_pdf(pages, new_file_name, destination_dir):
    try:
        new_pdf = pikepdf.Pdf.new()
        new_pdf.pages.extend(pages)
        dest_dir_with_new_file_name = os.path.join(destination_dir, new_file_name)
        new_pdf.save(dest_dir_with_new_file_name)
        return True  # Return True if the file was saved successfully
    except Exception as e:
        print(f"Error occurred while creating and saving PDF: {str(e)}")
        return False  # Return False if an error occurred

def get_new_file_name(regex_num, today, total_target_amt, company_name):
    # File naming convention for EXXONMOBIL ETFs only; wrapping amount in () to indicate amount owed
    if company_name == 'EXXONMOBIL' and re.match(r'ETF-\d+', regex_num):
        new_file_name = f'{regex_num}-{today}-({total_target_amt}).pdf'

    # File naming convention for loyalty files
    elif re.match(r'LRD-\d+', regex_num):
        new_file_name = f'{today}-Loyalty.pdf'

    # File naming convention for chargebacks/retrievals
    elif (re.match(r'CBK-\d+', regex_num) or re.match(r'RTV-\d+', regex_num)):
        new_file_name = f'{today}-CHARGEBACK REQUEST.pdf'

    # File naming convention for all other files (CCM, CMB, non-EXXON ETFs)
    else:
        new_file_name = f'{regex_num}-{today}-{total_target_amt}.pdf'
    # print(f'new_file_name: {new_file_name}')
    return new_file_name


In [40]:
def process_multi_page(pdf, page_num, company_names, regex_patterns, company_name_to_company_subdir_mapping):
    current_page_text = extract_text_from_pdf_page(pdf.pages[page_num])
    print(f'Processing page: {page_num + 1}')
    # print(f'\n*****************************\n{current_page_text}\n*****************************\n')

    # for company_name in company_names:
    #     # Handles CCM, CMB, LRD multi page docs
    #     if company_name in current_page_text and 'END MSG' not in current_page_text:
    #         print(f'\n*****************************\n{current_page_text}\n*****************************\n')
    #         for pattern in regex_patterns:
    #             if re.search(pattern, current_page_text):
    #                 print(f'process_multi_page - page_num-----------: {page_num + 1}')
    #                 # print(f'For company {company_name}\nand regex_pattern {pattern}\nOn page {page_num + 1} searching in text\n*****************************\n {current_page_text}\n*****************************\n')
    #
    #                 current_pages = []
    #                 current_page_texts = []

                    # while 'END MSG' not in current_page_text and page_num < len(pdf.pages):
                    #     current_pages.append(pdf.pages[page_num])
                    #     current_page_text = extract_text_from_pdf_page(pdf.pages[page_num])
                    #     current_page_texts.append(current_page_text)
                    #     print(f'current_page_texts: {current_page_texts}')
                    #
                    #     print(f'@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ {page_num + 1}')
                    #     page_num += 1
                    #     print(f'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!{page_num + 1}')
                    #
                    #     if page_num >= len(pdf.pages):
                    #         break
                    #
                    # current_page_text = "".join(current_page_texts)
                    #
                    # regex_num, today, total_amount = extract_info_from_text(current_page_text, pattern)
                    #
                    # new_file_name = get_new_file_name(regex_num, today, total_amount, company_name)
                    # print(f'\nnew_file_name: {new_file_name}')
                    # destination_dir = company_name_to_company_subdir_mapping[company_name]
                    #
                    # if company_name == 'CONCORD FIRST DATA RETRIEVAL' and re.match(r'CMB-\d+', regex_num):
                    #     multi_page_pdf_saved = create_and_save_pdf(current_pages, new_file_name, destination_dir)
                    #     if multi_page_pdf_saved:
                    #         print("###################################################################")
                    #
                    # # POST PROCESSING ONLY FOR EXXON CCMs  'TOTAL DISTRIBUTOR'
                    # elif company_name == 'EXXONMOBIL' and re.match(r'CCM-\d+', regex_num):
                    #     multi_page_pdf_saved_in_temp = create_and_save_pdf(current_pages, new_file_name, temp_dir)
                    #
                    # # Loyalty Reward Detail (LRD) files
                    # elif re.match(r'LRD-\d+', regex_num):
                    #     loyalty_pdf_saved = create_and_save_pdf(current_pages, new_file_name, destination_dir)

    return page_num


In [41]:
def process_pages(filepath, company_name_to_company_subdir_mapping, company_names, regex_patterns, is_multi_page):
    try:

        # Read original PDF from dls dir
        print(f'Processing file: {filepath}')
        with pikepdf.open(filepath) as pdf:
            page_num = 0  # Initialize page_num
            while page_num < len(pdf.pages):
                print(f'page_num: {page_num + 1}')

                # Process pages and update the page number at original PDF (macro) level
                if is_multi_page:
                    new_page_num = process_multi_page(pdf, page_num, company_names, regex_patterns, company_name_to_company_subdir_mapping)
                else:
                    new_page_num = process_single_page(pdf, page_num, company_names, regex_patterns, company_name_to_company_subdir_mapping)

                # if process_page has not incremented
                # prevents one off issue
                if new_page_num == page_num:
                    page_num += 1
                else:
                    page_num = new_page_num

            # If all pages processed without errors, return True
            return True
    except Exception as e:
        # If any error occurred, print it and return False
        print(f'An unexpected error occurred: {str(e)}')
        return False


In [43]:
def process_pdfs(filepath, company_name_to_company_subdir_mapping, company_names, regex_patterns):
    try:
        # print(f'Processing all single-page CCMs....\n')
        # single_pages_processed = process_pages(filepath, company_name_to_company_subdir_mapping, company_names, regex_patterns, is_multi_page=False)
        # if single_pages_processed:
        #     print(f'successfully finished processing all single page CCMs\n')

        print(f'Now processing all multi-page CCMs....\n')
        multi_pages_processed = process_pages(filepath, company_name_to_company_subdir_mapping, company_names, regex_patterns, is_multi_page=True)
        if multi_pages_processed:
            print(f'successfully finished processing all multi paged CCMs\n')

        # once all single and multi page CCMs for EXXON are done,
        # post process all pdfs in temp dir
        if multi_pages_processed:
        # if single_pages_processed and multi_pages_processed:
        #     Exxon Temp dir post processing
            print(f'Post processing for EXXON CCMs.....')
            merge_rename_and_summate(temp_dir)

    except Exception as e:
        print(f'An error occurred: {str(e)}')


process_pdfs(file_path, company_name_to_subdir_full_path_mapping_credit_cards, company_names, regex_patterns)

Now processing all multi-page CCMs....

Processing file: /Users/ekim/workspace/txb/docs/ccm_full.pdf
page_num: 1
********[pdf_stream]********************: <_io.BytesIO object at 0x7fac2822ac50>
Processing page: 1
page_num: 2
********[pdf_stream]********************: <_io.BytesIO object at 0x7fac2822b6f0>
Processing page: 2
page_num: 3
********[pdf_stream]********************: <_io.BytesIO object at 0x7fac18e4e070>
Processing page: 3
page_num: 4
********[pdf_stream]********************: <_io.BytesIO object at 0x7fac281016c0>
Processing page: 4
page_num: 5
********[pdf_stream]********************: <_io.BytesIO object at 0x7fabf8705a80>
Processing page: 5
page_num: 6
********[pdf_stream]********************: <_io.BytesIO object at 0x7fabf8707650>
Processing page: 6
page_num: 7
********[pdf_stream]********************: <_io.BytesIO object at 0x7fabf8707880>
Processing page: 7
page_num: 8
********[pdf_stream]********************: <_io.BytesIO object at 0x7fabf8954360>
Processing page: 8
pag

KeyboardInterrupt: 